## 01. 데이터 불러오기

In [3]:
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
    
matplotlib.rcParams['axes.unicode_minus'] = False

%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [8]:
import pandas as pd

train = pd.read_csv("C:/Users/myhkj/GitHub/Dacon_ParkingDemand/train_df_errno.csv")
test = pd.read_csv("C:/Users/myhkj/GitHub/Dacon_ParkingDemand/test_df.csv")
sub = pd.read_csv("C:/Users/myhkj/GitHub/Dacon_ParkingDemand/sample_submission.csv")
age = pd.read_csv("C:/Users/myhkj/GitHub/Dacon_ParkingDemand/age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/myhkj/GitHub/Dacon_ParkingDemand/test_df.csv'

In [ ]:
train.columns

In [ ]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']

## 02. 결측치를 처리(1)

### 데이터 결합

In [ ]:
all_df = pd.concat([train, test], join='inner')
all_df

In [ ]:
all_df.isnull().sum()

### 자격유형(test) 결측치 처리

In [ ]:
all_df['지역'].unique()

In [ ]:
all_df.loc[all_df['자격유형'].isnull()]

In [ ]:
grouped = all_df.groupby(['단지코드', '임대건물구분', '지역','공급유형'])
group1 = grouped.get_group( ('C2411', '아파트', '경상남도', '국민임대')  )
group1

In [ ]:
group2 = grouped.get_group( ('C2253', '아파트', '강원도', '영구임대')  )
group2

In [ ]:
all_df.loc[ 196, "자격유형"] = 'A'
all_df.loc[ 258, "자격유형"] = 'C'

In [ ]:
print(all_df.자격유형.unique())

In [ ]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }

all_df['자격유형'] =all_df['자격유형'].map(mapping).astype(int)

In [ ]:
print(all_df.공급유형.unique())

In [ ]:
all_df.columns

### 10분내버스정류장수 (tr) 결측치 처리

In [ ]:
all_df.isnull().sum()

In [ ]:
all_df.loc[ train['10분내버스정류장수'].isnull(), :]

In [ ]:
all_df['임대건물구분'].unique()

In [ ]:
pd.set_option("display.max_rows", 100)
pd.get_option("display.max_rows")

In [ ]:
grouped = train.groupby(['임대건물구분', '지역'])
group1 = grouped.get_group( ('아파트', '경상남도')  )
group1

In [ ]:
# 데이터 확인 후, 임의 처리 4
all_df.loc[ all_df['10분내버스정류장수'].isnull(), "10분내버스정류장수"] = 4

In [ ]:
all_df.loc[ all_df['10분내버스정류장수'].isnull(), :]

In [ ]:
all_df.info()

In [ ]:
all_df.head()

### 라벨 인코딩

In [ ]:
for c in all_df.columns:
    print(all_df[c].unique())

In [ ]:
all_df.unique()

In [ ]:
gubun1 = {'아파트':1, '상가':2}
gubun2 = {'경상남도':1, '대전광역시':2, '경기도':3, '전라북도':4, '강원도':5,
          '광주광역시':6, '충청남도':7, '부산광역시':8, '제주특별자치도':9, '울산광역시':10,
          '충청북도':11, '전라남도':12, '경상북도':13, '대구광역시':14, '서울특별시':15, '세종특별자치시':16}

gubun3 = {'국민임대':1, '공공임대(50년)':2, '영구임대':3, '임대상가':4, '공공임대(10년)':5, 
          '공공임대(분납)':6, '장기전세':7, '공공분양':8, '행복주택':9, '공공임대(5년)':10}

all_df['임대건물구분_lbl'] = all_df['임대건물구분'].map(gubun1)
all_df['지역_lbl'] = all_df['지역'].map(gubun2)
all_df['공급유형_lbl'] = all_df['공급유형'].map(gubun3)

all_df

In [ ]:
all_df.단지코드.unique()

In [ ]:
all_df.info()

In [ ]:
all_df['단지코드'] = all_df['단지코드'].astype("category")

In [ ]:
all_df['단지코드_lbl'] = all_df['단지코드'].cat.codes
all_df

In [ ]:
all_df_last = all_df.drop(['임대건물구분', '지역', '공급유형'] , axis=1)
all_df_last

In [ ]:
for c in all_df_last.columns:
    print(all_df_last[c].unique())

In [ ]:
train.shape, test.shape

In [ ]:
train_df = all_df_last.iloc[0:2896,:]
test_df = all_df_last.iloc[2896:,:]

train_df.shape, test_df.shape

train_df = pd.concat([train_df, train['등록차량수'] ], axis=1)
train_df

In [ ]:
train_df.columns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df.corr()['등록차량수']

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train_df.corr(), annot=True, fmt=".2f")

In [ ]:
sel = [ '총세대수', '전용면적', '전용면적별세대수', '공가수', '자격유형', '10분내버스정류장수', 
       '단지내주차면수', '임대건물구분_lbl', '지역_lbl', '공급유형_lbl',
       '단지코드_lbl']

X = train_df[sel]
y = train_df['등록차량수']
test_X = test_df[sel]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

In [ ]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

In [ ]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

In [ ]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

In [ ]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
# 기본 옵션 확인
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 0.33, # 각나무마다 사용하는 feature 비율
            learning_rate = 0.1,
            max_depth = 5, 
            alpha = 0.1, 
            n_estimators = 2500)  # n_estimators=100
xg_reg.fit(X_train, y_train)
pred = xg_reg.predict(X_test)

print("학습(score) :", xg_reg.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", xg_reg.score(X_test, y_test) ) # 결정계수

In [ ]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

In [ ]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(test_X)
pred[0:10]

In [ ]:
test_df['등록차량수'] = pred
test_df['단지별차량수평균'] = test_df.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test_df.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

In [ ]:
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

In [ ]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

In [ ]:
sub_df.to_csv('fourth_rf_0714.csv', index=False)
sub_df.head()

In [ ]:
import os
os.listdir(os.getcwd())